In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **I will further manipulate this code using the data Mo created, I was supposed to now do it on the original data I had but it would be counterproductive. This draft was based on a simple dataset I created just to see that the code works as intended.**

# <center> Model Fitting </center>
For the purpose of this study, we fit the data to OLS regressions. We first explain what these models are and subsequently elaborate how we use them.<br> 

Multiple linear regression<br>
This kind of regression model is used to analyse a linear relationship between response variable $Y$ and covariates $X_i$. Multiple regression model considers two or more explanatory variables:
$$ y_i = \beta_0 + \beta_1x_{1,i} + \beta_2x_{2,i}+ \dots + \beta_mx_{m,i} + \epsilon_i $$ 
The coefficients $\beta_{1},\dots,\beta_{k}$ measure the effect of each individual predictor after taking into account the effects of all the other predictors. The error term $\epsilon_i$ reflects random variation in $Y$ from variables other than $x$. 
With this regression modeling, estimate the regression function:
$$ \hat{r}(x) = \hat{\beta_0} + \hat{\beta_1}x $$

Categorical Predictors<br>
A regression that includes a categorical predictor with $n$ categories has the equation:
$$ y_i = \beta_0 + \beta_1 x_{1,i} + \beta_{I1} I_{1,i} + \beta_{I2} I_{2,i} + \dots + \beta_{I(n-1)} I_{n-1,i} \epsilon_i $$
where $x_{1,i}$ and $y_i$ are continuous variables and $I_{2,i}$ is an indicator variable.
The model is interpreted as a linear regression model that has an intercept that depends on the category of the individual.

Assumptions:<br>
error term $\epsilon_i$ is assumed to be a normal random variable<br>
$r(x)$ is linear. <br>

We use data on household monthly income, salaries_1 and salaries_2 (these are encoded variables which tells us the if the household main source of income is salaries or not) to fit the multiple linear regression. To analyse the relationship between these variables we come across issues such an extremly high condition number indicating that there is multicollinearity in with the predictors. We also have a low adjusted R squared.

In [ ]:
fit1 = smf.ols(formula="mon_income ~ salaries_1+salaries_2", data=full_data_dummies).fit(fit_intercept=False)

print(fit1.summary())


In [ ]:
without_response = full_data_dummies.drop(['mon_income','year'],axis=1)
response='mon_income'
formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(without_response.columns))

fit3 = smf.ols(formula, data = full_data_dummies).fit(fit_intercept=False)
print(fit3.summary())

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(full_data_dummies, test_size=0.2)



## Variable and Model Selection

From our given dataset, there are a large number of variables to choose from. As we have seen from the above models, including all variables we run into multicollinearity. Including a restricted number we may have subjective models. Thus the choice of which model to include is dependent on the predictive ability of the entire model.
 Common measures to assess the predictive ability of the model are:
* AIC, AICc, BIC 
* $R^2$ and adj-$R^2$ 


#### Stepwise Regression
Since we have a large number of potential predictors, there are a lot of subsets to go through. Thus, we use stepwise regression procedures. These procedures find a good model, but it may not be the best model.

1) Check for multicollinearity:
This is when the predictores are linearly dependent, thus at least one predictor is redundatant. This causes problems as we have unstable samples. We employ Variance Inflation Factors(VIF) to check for multicollinearity.Interpretation of the VIF is  variance of coefficient estimator is inflated by a factor of VIF in comparison to if the variable was not correlated. We will drop thenredundant feature

2) Backward Regression: 
start with all variables and measure the p value, then remove at a variable time and measure p value. Then choose model with best pvalue and iterate through the last two steps 

3) Forward Regression
start with model that has only the intercept then include one variable at a time and measure adjusted $R^2$. Choose model with best $R^2$ and iterate through the last two steps

In [ ]:
def multicol(x, thresh=10):
    x = x.drop(['mon_income'],axis=1)
    variables=list(range(x.shape[1]))
    dropped= True
    while dropped:
        dropped=False
        vif = [variance_inflation_factor(x.iloc[:,variables].values, ix) for ix in variables]
        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + x.iloc[:,variables].columns[maxloc] + '\' at index: ' + str(maxloc))
            x.drop(x.columns[variables[maxloc]],1, inplace=True)
            variables = list(range(x.shape[1]))
            dropped=True  
    t = x.columns[variables]
    return t

In [ ]:
def multicollinearity(x, sample_size, thresh=10):
    sample_train=resample(x,n_samples=sample_size)
    remain=multicol(sample_train)
    return remain

In [ ]:
remaining=multicollinearity(train, sample_size=1000, thresh=5)
remaining

In [ ]:
X = train[remaining]
y = train['mon_income']

In [ ]:
def backward_selction(x,y):
    cols= list(X.columns)
    pmax = 1
    verbose=True
    while (len(cols)>0):
        p=[]
        x_1=pd.DataFrame(X[cols])
        x_1= sm.add_constant(x_1)
        model= sm.OLS(y,x_1).fit()
        p= pd.Series(model.pvalues.values[1:],index=cols)
        pmax= max(p)
        feature_with_p_max=p.idxmax()
        if(pmax>0.05):
            cols.remove(feature_with_p_max)
            print('Drop {:30} with p-value {:.6}'.format(feature_with_p_max, pmax))
        else:
            break
    selected_features= cols
    return selected_features
selected_features=backward_selction(X,y)
selected_features

In [ ]:
ss=train[selected_features]
selected_data= ss.merge(y.to_frame(), left_index=True, right_index=True)

In [ ]:
import statsmodels.formula.api as smf

def forward_selected(data):
    
    response='mon_income'
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_r= 0.0
    best_r = 0.0
    count=0
    
    while remaining and current_r == best_r:
        scores_with_candidates = []
        for variable in remaining:
             
            formula = "{} ~ {} + 1".format(response,' + '.join(selected + [variable]))
            score = smf.ols(formula, data).fit(fit_intercept=False).rsquared_adj
            scores_with_candidates.append((score, variable))
        count+=1
        scores_with_candidates.sort()
        best_r, best_candidate = scores_with_candidates.pop()
        if ((current_r < best_r) or (count>=20)):
            print('Adding: {}'.format(best_candidate))
            count=0
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_r = best_r
    formula = "{} ~ {} + 1".format(response,' + '.join(selected))
    model = smf.ols(formula, data).fit(fit_intercept=False)
    return model

In [ ]:
model=forward_selected(selected_data)

Checking if the remaining predictors contribute to the overall model. We employ adjusted R square for this and we see that all the predictors are included in the optial model. That is if they are all jointly significant. As we have mentioned that while these procedures produces a good model but it might not be the optimal one. Thus we proceed to coduct bootstrapping of those steps to furture refine the chosen model and features.

In [ ]:
def bootstrapping(data,simulations, sample_size):
    optimal_models=[]
    
    for i in range(0,simulations):
        
        print('Starting iteration: {:}'.format(i))
        
        data_sampled=resample(data,n_samples=sample_size)
        remaining=multicollinearity(data_sampled, sample_size=sample_size, thresh=5)
    
        
        X = train[remaining]
        y = train['mon_income']

        result = backward_selction(X, y) 
        result.append('mon_income')
        
        model=forward_selected(data_sampled[result])
        choosen=[]
        choosen=[model,model.params,model.bse,model.pvalues,model.conf_int(),model.condition_number,model.rsquared_adj]
        optimal_models.append(choosen)
        print('Finish iteration: {:}'.format(i))
    return optimal_models


In [ ]:
selected_dataa=bootstrapping(train,simulations=300, sample_size=5000)

In [ ]:
def common_features(selected_dataa,adj_r,iters_fraction):
    features = {}
    for i in range(len(selected_dataa)):
        if (selected_dataa[i][6]>adj_r):
            feat=list(selected_dataa[i][1].index)
            for f in feat:
                try:
                    features[f]+=1
                except:
                    features[f]=1
    valid=[]
    for feat,value in features.items():
        if ((value>=iters_fraction) and (feat!='Intercept')):
            valid.append(feat)
    return valid

In [ ]:
selected_data1=common_features(selected_dataa,0.4,250)
selected_data1


### Residual Analysis

Now that we have selected the optimal model, we analyse the residuals and check if we met the assumption of the model. 
We perform the following four plots to check for the following:<br>
1) heteroskedasticity<br>
2) if residuals are related to predictors<br>
3)  if residuals are related to dependent variable<br>

If these assuptions are not satisfied the the least square estimate is not efficient. The estimated will be biased at the statistical tests will be misleading.


In [ ]:
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize']=10000

# figure window
fig = plt.figure(figsize=[16,16]);
    # subplots
ax1 = plt.subplot2grid((2,2), (0,0), colspan=2)


response='mon_income'
formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected_data1))
fit3 = smf.ols(formula, data=train).fit(fit_intercept=False)
#residual plot
fit3.resid.plot(ax=ax1)
plt.suptitle('Residuals from Linear regression model', size = 20);
plt.subplots_adjust(top=0.9)

In [ ]:
from statsmodels.graphics.tsaplots import month_plot, plot_acf, plot_pacf
# figure window
fig = plt.figure(figsize=[10,6]);
ax1 = plt.subplot2grid((2,2), (0,0), colspan=2)
# fit regression model
response='mon_income'
formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected_data1))
fit3 = smf.ols(formula, data=train).fit(fit_intercept=False)
# ACF
plot_acf(fit3.resid, ax=ax1, lags=50, zero=False)    
plt.subplots_adjust(top=0.9)


In [ ]:
# figure window
fig = plt.figure(figsize=[10,6]);

# fit regression model
response='mon_income'
formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected_data1))
fit3 = smf.ols(formula, data=train).fit(fit_intercept=False)

# residual histogram
sns.distplot(fit3.resid, kde=True, rug=True, hist = True)
# tight format
fig.tight_layout()

In [ ]:
ax = plt.scatter(y = fit3.resid, x = fit3.predict())
plt.xlabel('Fitted');
plt.ylabel('Residuals');
plt.show()


In [ ]:
response='mon_income'
test_data=test

formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected_data1))
model = smf.ols(formula, test_data).fit()
print(model.summary())
